# ERCOT Price Forecasting Feature Analysis and Model Comparison

This script demonstrates how to analyze features that impact ERCOT electricity prices
and compares different model configurations to find the optimal setup.

In [4]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Union, Any
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Add the project directory to the path so we can import modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

plt.style.use('ggplot')
%matplotlib inline

from src.data.ercot_price_data import ErcotPriceData
from src.data.ercot_weather_data import ErcotWeatherData
from src.models.hybrid_model import HybridModel
from src.utils.preprocessing import (
    align_time_series,
    create_time_features,
    create_lag_features,
    create_rolling_features,
    prepare_data_for_model
)
from src.visualization.plotting import (
    plot_price_forecast,
    plot_volatility_forecast,
    plot_price_components,
    plot_model_performance
)

AttributeError: module 'pandera' has no attribute 'SchemaModel'

## Data Loading and Preparation

First, let's load the price and weather data for multiple ERCOT hubs.

In [ ]:
# Define date range for our analysis
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # Use one year of data
test_start_date = end_date - timedelta(days=30)  # Last 30 days for testing

# Define the price nodes to analyze
price_nodes = ['HB_HOUSTON', 'HB_NORTH', 'HB_SOUTH', 'HB_WEST']
locations = ['Houston', 'Dallas', 'San Antonio', 'Midland']

# Create empty dictionaries to store data for each node
price_data_dict = {}
weather_data_dict = {}

# Load price and weather data for each node/location
for node, location in zip(price_nodes, locations):
    # Load price data
    price_data_dict[node] = ErcotPriceData().load_data(
        start_date=start_date,
        end_date=end_date,
        price_node=node,
        resample_freq='H'
    )
    
    # Load weather data
    weather_data_dict[location] = ErcotWeatherData().load_data(
        start_date=start_date,
        end_date=end_date,
        location=location,
        resample_freq='H'
    )
    
    print(f"Loaded data for {node} / {location}")
    print(f"  Price data shape: {price_data_dict[node].shape}")
    print(f"  Weather data shape: {weather_data_dict[location].shape}")

## Price Comparison Across ERCOT Hubs

Let's compare price patterns across different ERCOT hubs.

In [ ]:
# Create a DataFrame with prices from all nodes
all_prices = pd.DataFrame()
for node in price_nodes:
    all_prices[node] = price_data_dict[node]['price']

# Plot price comparison
fig = go.Figure()

for node in price_nodes:
    fig.add_trace(go.Scatter(
        x=all_prices.index,
        y=all_prices[node],
        mode='lines',
        name=node
    ))

fig.update_layout(
    title='ERCOT Price Comparison Across Hubs',
    xaxis_title='Date',
    yaxis_title='Price ($/MWh)',
    height=600,
    width=1000,
    template='plotly_white',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()

# Calculate correlation between prices at different hubs
price_correlation = all_prices.corr()

# Plot correlation heatmap
fig_corr = px.imshow(
    price_correlation,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    title='Price Correlation Across ERCOT Hubs',
    height=500,
    width=700
)
fig_corr.show()

## Feature Engineering and Analysis

Let's analyze features for predicting electricity prices.

In [ ]:
# Select Houston hub for feature analysis
hub = 'HB_HOUSTON'
location = 'Houston'
price_data = price_data_dict[hub]
weather_data = weather_data_dict[location]

# Align and preprocess data
preprocessed_data = align_time_series(price_data, weather_data)
preprocessed_data = create_time_features(preprocessed_data)
preprocessed_data = create_lag_features(
    preprocessed_data, 
    target_column='price', 
    lag_periods=[1, 2, 3, 6, 12, 24, 48, 72, 168]  # Multiple lag periods for analysis
)
preprocessed_data = create_rolling_features(
    preprocessed_data, 
    target_column='price',
    windows=[24, 48, 72, 168],  # Multiple windows for analysis
    functions=['mean', 'std', 'min', 'max']  # Additional statistical features
)

# Drop NaN values
preprocessed_data = preprocessed_data.dropna()
print(f"Preprocessed data shape: {preprocessed_data.shape}")
print(f"Number of features: {preprocessed_data.shape[1] - 1}")  # Excluding price column

# Split into features and target
X = preprocessed_data.drop('price', axis=1)
y = preprocessed_data['price']

## Feature Importance Analysis

Let's calculate feature importance using mutual information.


In [ ]:
# Calculate mutual information between each feature and the price
mi_scores = mutual_info_regression(X, y)
mi_df = pd.DataFrame({'Feature': X.columns, 'MI Score': mi_scores})
mi_df = mi_df.sort_values('MI Score', ascending=False).reset_index(drop=True)

# Display top 20 features
print("Top 20 features by mutual information:")
print(mi_df.head(20))

# Plot feature importance
fig = px.bar(
    mi_df.head(20),
    x='MI Score',
    y='Feature',
    orientation='h',
    title='Feature Importance by Mutual Information',
    height=600,
    width=900
)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

## Feature Correlation Analysis

Let's analyze correlation between features and price.

In [ ]:
# Calculate correlation with price
corr_with_price = preprocessed_data.corr()['price'].drop('price')
corr_df = pd.DataFrame({'Feature': corr_with_price.index, 'Correlation': corr_with_price.values})
corr_df = corr_df.sort_values('Correlation', ascending=False).reset_index(drop=True)

# Display top positive and negative correlations
print("Top 10 features with positive correlation:")
print(corr_df.head(10))
print("\nTop 10 features with negative correlation:")
print(corr_df.tail(10))

# Plot correlations
fig = px.bar(
    pd.concat([corr_df.head(10), corr_df.tail(10)]).sort_values('Correlation'),
    x='Correlation',
    y='Feature',
    orientation='h',
    title='Feature Correlation with Price',
    color='Correlation',
    color_continuous_scale='RdBu_r',
    height=600,
    width=900
)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

## Time-Based Feature Analysis

Let's analyze how price patterns vary by time of day, day of week, and month.

In [ ]:
# Create time-based aggregations
hourly_avg = preprocessed_data.groupby('hour')['price'].mean()
daily_avg = preprocessed_data.groupby('day_of_week')['price'].mean()
monthly_avg = preprocessed_data.groupby('month')['price'].mean()

# Plot hourly price patterns
fig_hourly = px.line(
    x=hourly_avg.index,
    y=hourly_avg.values,
    title='Average Price by Hour of Day',
    labels={'x': 'Hour of Day', 'y': 'Average Price ($/MWh)'},
    height=400,
    width=800
)
fig_hourly.update_layout(template='plotly_white')
fig_hourly.show()

# Plot daily price patterns
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
fig_daily = px.bar(
    x=days,
    y=[daily_avg.get(i, 0) for i in range(7)],
    title='Average Price by Day of Week',
    labels={'x': 'Day of Week', 'y': 'Average Price ($/MWh)'},
    height=400,
    width=800
)
fig_daily.update_layout(template='plotly_white')
fig_daily.show()

# Plot monthly price patterns
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
fig_monthly = px.bar(
    x=months,
    y=[monthly_avg.get(i+1, 0) for i in range(12)],
    title='Average Price by Month',
    labels={'x': 'Month', 'y': 'Average Price ($/MWh)'},
    height=400,
    width=800
)
fig_monthly.update_layout(template='plotly_white')
fig_monthly.show()

## Weather Impact Analysis

Let's analyze how weather variables impact electricity prices.

In [ ]:
# Select key weather variables
weather_vars = ['temperature', 'wind_speed', 'solar_irradiance', 'humidity']

# Create scatter plots for each weather variable
for var in weather_vars:
    fig = px.scatter(
        preprocessed_data,
        x=var,
        y='price',
        title=f'Price vs {var.capitalize()}',
        trendline='ols',  # Add trend line
        opacity=0.5,
        height=500,
        width=800
    )
    fig.update_layout(template='plotly_white')
    fig.show()

# Calculate correlation matrix for weather variables and price
weather_price_corr = preprocessed_data[weather_vars + ['price']].corr()
fig_corr = px.imshow(
    weather_price_corr,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    title='Correlation: Weather Variables and Price',
    height=500,
    width=700
)
fig_corr.show()

## Model Hyperparameter Comparison

Let's compare different hyperparameter configurations for our hybrid model.

In [ ]:
# Define hyperparameter combinations to test
seq_lengths = [24, 48, 72]
forecast_horizons = [12, 24]
garch_p_values = [1, 2]
garch_q_values = [1, 2]

# Prepare data for model training/evaluation
train_data = preprocessed_data[preprocessed_data.index < test_start_date]
test_data = preprocessed_data[preprocessed_data.index >= test_start_date]

train_price = train_data[['price']]
test_price = test_data[['price']]

feature_columns = [col for col in train_data.columns if col != 'price']
train_features = train_data[feature_columns]
test_features = test_data[feature_columns]

# Initialize results tracking
results = []

# Train and evaluate models with different hyperparameters
for seq_length in seq_lengths:
    for forecast_horizon in forecast_horizons:
        for p in garch_p_values:
            for q in garch_q_values:
                print(f"\nTraining model with seq_length={seq_length}, forecast_horizon={forecast_horizon}, p={p}, q={q}")
                
                # Initialize model
                model = HybridModel(
                    seq_length=seq_length,
                    forecast_horizon=forecast_horizon,
                    p=p,
                    q=q,
                    mean='Zero',
                    vol='GARCH',
                    dist='normal'
                )
                
                # Train model
                model.fit(
                    price_data=train_price,
                    weather_data=train_features,
                    nn_epochs=20,  # Reduced for quicker comparison
                    nn_batch_size=32,
                    nn_validation_split=0.2,
                    verbose=0
                )
                
                # Make predictions
                forecasts = model.predict(
                    price_data=test_price,
                    weather_data=test_features,
                    confidence_level=0.95
                )
                
                # Calculate metrics
                actual = test_price['price']
                predicted = forecasts['price_forecast']
                
                mse = mean_squared_error(actual, predicted)
                rmse = np.sqrt(mse)
                mae = mean_absolute_error(actual, predicted)
                mape = np.mean(np.abs((actual - predicted) / actual)) * 100
                r2 = r2_score(actual, predicted)
                
                # Calculate confidence interval coverage
                lower_bound = forecasts['lower_bound']
                upper_bound = forecasts['upper_bound']
                within_bounds = ((actual >= lower_bound) & (actual <= upper_bound))
                coverage = within_bounds.mean() * 100
                
                # Store results
                results.append({
                    'seq_length': seq_length,
                    'forecast_horizon': forecast_horizon,
                    'garch_p': p,
                    'garch_q': q,
                    'MSE': mse,
                    'RMSE': rmse,
                    'MAE': mae,
                    'MAPE': mape,
                    'R2': r2,
                    'CI_Coverage': coverage
                })
                
                print(f"  RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2f}%, R2: {r2:.4f}, CI Coverage: {coverage:.2f}%")

# Convert results to DataFrame
results_df = pd.DataFrame(results)
print("\nModel comparison results:")
print(results_df)

## Visualizing Model Comparison Results

In [ ]:
# Plot RMSE by hyperparameters
fig = px.scatter(
    results_df,
    x='seq_length',
    y='RMSE',
    size='forecast_horizon',
    color='garch_p',
    symbol='garch_q',
    title='Model Performance Comparison (RMSE)',
    labels={'seq_length': 'Sequence Length (hours)', 'RMSE': 'Root Mean Squared Error'},
    height=500,
    width=800
)
fig.update_layout(template='plotly_white')
fig.show()

# Plot MAPE by hyperparameters
fig = px.scatter(
    results_df,
    x='seq_length',
    y='MAPE',
    size='forecast_horizon',
    color='garch_p',
    symbol='garch_q',
    title='Model Performance Comparison (MAPE)',
    labels={'seq_length': 'Sequence Length (hours)', 'MAPE': 'Mean Absolute Percentage Error (%)'},
    height=500,
    width=800
)
fig.update_layout(template='plotly_white')
fig.show()

# Plot CI Coverage by hyperparameters
fig = px.scatter(
    results_df,
    x='seq_length',
    y='CI_Coverage',
    size='forecast_horizon',
    color='garch_p',
    symbol='garch_q',
    title='Confidence Interval Coverage Comparison',
    labels={'seq_length': 'Sequence Length (hours)', 'CI_Coverage': 'CI Coverage (%)'},
    height=500,
    width=800
)
fig.update_layout(
    template='plotly_white',
    yaxis=dict(range=[80, 100])  # Focus on the relevant range for coverage
)
fig.show()

## Finding the Best Model Configuration

In [ ]:
# Find the model with lowest RMSE
best_rmse_idx = results_df['RMSE'].idxmin()
best_rmse_model = results_df.iloc[best_rmse_idx]
print("\nBest model by RMSE:")
print(best_rmse_model)

# Find the model with lowest MAPE
best_mape_idx = results_df['MAPE'].idxmin()
best_mape_model = results_df.iloc[best_mape_idx]
print("\nBest model by MAPE:")
print(best_mape_model)

# Find the model with best confidence interval coverage
# (closest to the expected 95%)
results_df['CI_Coverage_Error'] = abs(results_df['CI_Coverage'] - 95)
best_coverage_idx = results_df['CI_Coverage_Error'].idxmin()
best_coverage_model = results_df.iloc[best_coverage_idx]
print("\nBest model by confidence interval coverage:")
print(best_coverage_model)

## Train and Visualize Best Model

In [ ]:
# Use the best RMSE model configuration
best_seq_length = int(best_rmse_model['seq_length'])
best_forecast_horizon = int(best_rmse_model['forecast_horizon'])
best_p = int(best_rmse_model['garch_p'])
best_q = int(best_rmse_model['garch_q'])

print(f"Training final model with best configuration: seq_length={best_seq_length}, forecast_horizon={best_forecast_horizon}, p={best_p}, q={best_q}")

# Initialize and train best model
best_model = HybridModel(
    seq_length=best_seq_length,
    forecast_horizon=best_forecast_horizon,
    p=best_p,
    q=best_q,
    mean='Zero',
    vol='GARCH',
    dist='normal'
)

best_model.fit(
    price_data=train_price,
    weather_data=train_features,
    nn_epochs=50,  # More epochs for final model
    nn_batch_size=32,
    nn_validation_split=0.2,
    verbose=1
)

# Generate forecasts
best_forecasts = best_model.predict(
    price_data=test_price,
    weather_data=test_features,
    confidence_level=0.95
)

# Plot final forecasts
fig_final = plot_price_forecast(
    forecasts=best_forecasts,
    historical_data=test_price,
    title=f"Best Model: ERCOT Houston Hub Price Forecast (seq={best_seq_length}, horizon={best_forecast_horizon}, GARCH({best_p},{best_q}))"
)
fig_final.show()

# Plot model performance
fig_performance = plot_model_performance(
    actual_prices=test_price['price'],
    forecasted_prices=best_forecasts['price_forecast'],
    train_test_split_date=test_start_date,
    title="Best Model Performance"
)
fig_performance.show()

In [ ]:
## Multi-Step Forecast Evaluation

Let's analyze how model performance changes as we forecast further into the future.

In [ ]:
# Extract multi-step forecasts for analysis
# We'll compare 1, 6, 12, and 24 hours ahead

# Use the best model to make a new set of forecasts
# for analysis of forecast horizons
horizon_forecasts = best_model.predict(
    price_data=test_price,
    weather_data=test_features,
    confidence_level=0.95
)

# Create DataFrame to store actual and forecasted values at different horizons
horizons_to_evaluate = [1, 6, 12, min(24, best_forecast_horizon)]
forecast_comparison = pd.DataFrame({'actual': test_price['price']})

# Extract forecasts at different horizons
for h in horizons_to_evaluate:
    if h <= best_forecast_horizon:
        horizon_col = f'h{h}'
        # Offset the forecast by h-1 steps to align with actuals
        forecast_comparison[horizon_col] = horizon_forecasts['price_forecast'].shift(-(h-1))

# Drop NaN values
forecast_comparison = forecast_comparison.dropna()

# Calculate metrics for each horizon
horizon_results = []
for h in horizons_to_evaluate:
    if h <= best_forecast_horizon:
        horizon_col = f'h{h}'
        h_mse = mean_squared_error(forecast_comparison['actual'], forecast_comparison[horizon_col])
        h_rmse = np.sqrt(h_mse)
        h_mae = mean_absolute_error(forecast_comparison['actual'], forecast_comparison[horizon_col])
        h_mape = np.mean(np.abs((forecast_comparison['actual'] - forecast_comparison[horizon_col]) / forecast_comparison['actual'])) * 100
        
        horizon_results.append({
            'Horizon (hours)': h,
            'RMSE': h_rmse,
            'MAE': h_mae,
            'MAPE': h_mape
        })

# Create DataFrame with horizon results
horizon_results_df = pd.DataFrame(horizon_results)
print("\nForecast error by horizon:")
print(horizon_results_df)

# Plot metrics by horizon
metrics = ['RMSE', 'MAE', 'MAPE']
for metric in metrics:
    fig = px.line(
        horizon_results_df,
        x='Horizon (hours)',
        y=metric,
        markers=True,
        title=f'{metric} by Forecast Horizon',
        height=400,
        width=700
    )
    fig.update_layout(template='plotly_white')
    fig.show()

# Plot multi-horizon forecasts
fig = go.Figure()

# Add actual prices
fig.add_trace(go.Scatter(
    x=forecast_comparison.index[-48:],  # Last 48 hours
    y=forecast_comparison['actual'][-48:],
    mode='lines',
    name='Actual Price',
    line=dict(color='black', width=2)
))

# Add forecasts at different horizons
colors = ['blue', 'green', 'orange', 'red']
for i, h in enumerate(horizons_to_evaluate):
    if h <= best_forecast_horizon:
        horizon_col = f'h{h}'
        fig.add_trace(go.Scatter(
            x=forecast_comparison.index[-48:],
            y=forecast_comparison[horizon_col][-48:],
            mode='lines',
            name=f'{h}-hour ahead',
            line=dict(color=colors[i % len(colors)])
        ))

fig.update_layout(
    title='Price Forecasts at Different Horizons (Last 48 Hours)',
    xaxis_title='Date',
    yaxis_title='Price ($/MWh)',
    height=500,
    width=900,
    template='plotly_white',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig.show()

## Conclusion

In this script, we've performed an extensive analysis of features that impact ERCOT electricity prices and compared different model configurations. Key findings:

1. Feature importance analysis identified the most predictive variables for price forecasting
2. We observed clear price patterns by time of day, day of week, and season
3. Weather variables showed significant correlations with electricity prices
4. Model hyperparameter tuning found the optimal configuration for our hybrid model
5. Performance evaluation across different forecast horizons showed how accuracy degrades with longer horizons

These insights help us better understand the drivers of ERCOT prices and how to optimize our forecasting approach. 